# Unet model
example to get the Unet architecture without the cropping described in the original paper

Reference code for architecture  
https://github.com/zhixuhao/unet/blob/master/model.py  
  
  
Reference model paper:  
https://arxiv.org/abs/1505.04597  

In [1]:
import numpy as np
import tensorflow as tf
import keras

/home/bsaldivar/anaconda3/envs/keras-gpu-35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [20]:
from keras.layers import *
from keras.models import *
from keras.optimizers import Adam,Adadelta

def get_name(prefix,layer_counter,scope="CNN"):
    #If layer_counter does not exist create it
    counter = layer_counter.get(prefix,0)
    output = scope+"_"+prefix+"_"+str(counter)
    counter+=1
    layer_counter[prefix]=counter
    return output

def get_unet(input_size = (256,256,3),output_channels = 1,lr = 0.01,decay = 1e-6,model_name="Unet",layers_depth = [64,128,256,512,1024]):
    lc = {}
    inputs = Input(input_size,name="Input")
    x = inputs
    scope="CNN"
    layers = []
    layers_depth_up = layers_depth[:-1]
    layers_depth_up.reverse()
    for layer_depth in layers_depth:
        for _ in range(2):
            x = Conv2D(layer_depth,3,padding = 'same',kernel_initializer = 'he_normal',name=get_name("Conv",lc,scope=scope))(x)
            x = BatchNormalization(name=get_name("batch_norm",lc,scope=scope))(x)
            x = Activation('relu',name=get_name("relu",lc,scope=scope))(x)
            if _ == 1:
                layers.append(x)
        if layer_depth!=layers_depth[-1]:
            x = MaxPooling2D(pool_size=(2, 2),name=get_name("MaxPool",lc,scope=scope))(x)
    for i,layer_depth_up in enumerate(layers_depth_up):
        x = UpSampling2D(size = (2,2),name=get_name("Upsampling",lc,scope=scope))(x)
        x = Concatenate(axis = 3)([layers[-(i+2)],x])
        for _ in range(2):
            x = Conv2D(layer_depth_up,3,padding = 'same',kernel_initializer = 'he_normal',name=get_name("Conv",lc,scope=scope))(x)
            x = BatchNormalization(name=get_name("batch_norm",lc,scope=scope))(x)
            x = Activation('relu',name=get_name("relu",lc,scope=scope))(x)
    #Make output
    x = Conv2D(output_channels,3,padding = 'same',kernel_initializer = 'he_normal',name=get_name("Conv",lc,scope=scope))(x)
    x = BatchNormalization(name=get_name("batch_norm",lc,scope=scope))(x)
    x = Activation('sigmoid',name="Output")(x)
    model = Model(inputs=inputs,outputs=x,name=model_name)
    optimizer=Adadelta(lr=lr, rho=0.95, epsilon=None, decay=decay)
    model.compile(optimizer=optimizer,loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model



unet = get_unet(input_size = (256,256,3),output_channels = 1,lr = 0.01,decay = 1e-6,model_name="Unet",layers_depth = [64,128,256,512,1024])


In [21]:
unet.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
CNN_Conv_0 (Conv2D)             (None, 256, 256, 64) 1792        Input[0][0]                      
__________________________________________________________________________________________________
CNN_batch_norm_0 (BatchNormaliz (None, 256, 256, 64) 256         CNN_Conv_0[0][0]                 
__________________________________________________________________________________________________
CNN_relu_0 (Activation)         (None, 256, 256, 64) 0           CNN_batch_norm_0[0][0]           
__________________________________________________________________________________________________
CNN_Conv_1